# Setup

📝 *Import required libraries and files:*

In [1]:
import pandas as pd
import numpy as np
from google_play_scraper import app, Sort, reviews_all
import time

# 1.1 - Fetch Google Play data

To **scrape data from Google Play** Store we will use `google_play_scraper`. 

We will collect data from **6 different UK retail banks**: Santander UK, HSBC, LLoyds, Barclays, Revolut and Monzo. While this is not an exhaustive list of the retail banking competitive landscape in UK, it provides a good understanding of the main players and balances traditional banks with digital-born banks such as Revolut and Monzo. 

The focus was to obtain **individual reviews** (e.g., score, review text, date, reply, etc) for each App that will be used as input to perform sentiment analysis and categorization of motives of (dis)satisfaction with the APP;

Each block of information will be collected using a for loop that fetches data for all the 6 banks.

In [2]:
def scrape_reviews():
    
    # List of APPs' packages to be scraped
    apps = {
        "Santander UK": "uk.co.santander.santanderUK",
        "HSBC": "uk.co.hsbc.hsbcukmobilebanking",
        "LLoyds": "com.grppl.android.shell.CMBlloydsTSB73",
        "Barclays": "com.barclays.android.barclaysmobilebanking",
        "Revolut": "com.revolut.revolut",
        "Monzo": "co.uk.getmondo"
    }

    # Loop through the apps and scrape reviews
    all_reviews = []

    for app_name, app_id in apps.items():
        start_time = time.time()
        try:
            reviews = reviews_all(
                app_id,
                sleep_milliseconds=100,  
                lang='en',
                country='gb'
            )
            for review in reviews:
                all_reviews.append({
                    "app_name": app_name,
                    "user_name": review["userName"],
                    "score": review["score"],
                    "text": review["content"],
                    "date": review["at"],
                    "thumbs_up": review["thumbsUpCount"],
                    "Reply":review['replyContent'],
                    'Reply_Date':review['repliedAt'],
                    'App_Version':review['appVersion']
                })
            
            elapsed = time.time() - start_time
            print(f"✅ Fetched reviews for {app_name} in {elapsed:.2f} seconds")
            
        except Exception as e:
            print(f"Error fetching reviews for {app_name}: {e}")
    
        time.sleep(2)

    # Save to CSV
    df = pd.DataFrame(all_reviews)
    filename = "1_df_raw.csv"
    path = fr"C:\Users\pedro\OneDrive\Escritorio\Projetos\Banking APPs Reviews\{filename}"
    df.to_csv(path, index=False)
    print(f"✅ Done! Reviews saved to {filename}")

    return df

In [3]:
df_raw = scrape_reviews()

✅ Fetched reviews for Santander UK in 48.62 seconds
✅ Fetched reviews for HSBC in 23.43 seconds
✅ Fetched reviews for LLoyds in 90.55 seconds
✅ Fetched reviews for Barclays in 170.30 seconds
✅ Fetched reviews for Revolut in 166.63 seconds
✅ Fetched reviews for Monzo in 12.07 seconds
✅ Done! Reviews saved to 1_df_raw.csv


# 1.2. - Data Understanding

In [4]:
print(f"df shape: {df_raw.shape}")
df_raw.info()

df shape: (813103, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 813103 entries, 0 to 813102
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   app_name     813103 non-null  object        
 1   user_name    813103 non-null  object        
 2   score        813103 non-null  int64         
 3   text         812978 non-null  object        
 4   date         813103 non-null  datetime64[ns]
 5   thumbs_up    813103 non-null  int64         
 6   Reply        111182 non-null  object        
 7   Reply_Date   111182 non-null  datetime64[ns]
 8   App_Version  747482 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(5)
memory usage: 55.8+ MB


In [5]:
df_raw['app_name'].value_counts()

app_name
Barclays        264091
Revolut         260691
LLoyds          147695
Santander UK     79934
HSBC             38644
Monzo            22048
Name: count, dtype: int64

Data was successfully collected and saved into `banking_app_reviews.csv` file with a **total of 813k reviews**, with individual values ranging from 22k reviews for Monzo up to 264k for Barclays. 

It is worth noticing that, despite the usage of `review_all` method, the **number of fetched reviews is below the total number of reviews reported in Google Play Store for each of these APPs**. This situation can be explained by several factors, from which we highlight three of them: 

1. **`reviews_all` only fetches “publicly accessible” reviews** - i.e., while the Play Store reports the total number of reviews, not all of them are retrievable via scraper since some of them are hidden, limited by location or language or removed over time;

2. **Country and language filters** - i.e., we're only feching data in english from UK based users, but there are reviews in other languages and from users based in other coutries (especially for universal APPs such as 'Revolut');

3. **Scraping limitations**: Google Play's paginated interface does not expose the total number of reviews in one go, and even though `review_all` is designed to loop until it gets all reviews, it usually achieves a certain limit. Apart from that, Google may intentionally limit the amount of content exposed to unofficial scraping tools as the one we used. 

Despite this, **the data we have collected is more than enough to extract meaningfull insights and derive conclusions**, since the primary objective of this project is not to extract the total number of reviews per se, but instead to generalize sentiment and key topics for each app.